# Solving the Model

In Tutorial 4, we went over how the model is generated when GenX is run using `Run.jl`. In the function `run_genx_case_simple` (or multistage), after `generate_model` is called, `solve_model` is called to solve the EP.

<img src="files/runcase.png" style="width: 805px; height: auto" align="left">

In this tutorial, we go over how to use JuMP to solve a model, what it looks like to solve GenX, and how to edit the solver settings.

### A Simple Example

From Tutorial 4, we have the model:

\begin{align}
& \min 10 x + 15 y &\text{Objective function (cost)}\\ 
& \text{s.t.} & \\
& x + y \geq 10 &\text{Grid Demand}\\
& 55x + 70y \leq \ 1000 &\text{Construction constraint}\\
& 40 x + 5 y \leq 200 &\text{Emissions constraint} \\
& x, y \geq 0 &\text{Non-negativity constraints}\\
\end{align}




In [1]:
using JuMP
using HiGHS

In [2]:
power = Model(HiGHS.Optimizer)

@variable(power,x,Int) # Coal
@variable(power,y,Int) # Wind

@constraint(power, non_neg_x, x >= 0) # Non-negativity constraint (can't have negative power plants!)
@constraint(power, non_neg_y, y >= 0) # Non-negativity constraint

@constraint(power, emissions, 40x + 5y <= 200) # Emisisons constraint
@constraint(power, construction_costs, 55x + 70y <= 1000) # Cost of constructing a new plant

@constraint(power, demand, x + y >= 10) # Grid demand

@expression(power,objective,10x+15y)

@objective(power, Min, objective)

10 x + 15 y

JuMP uses the function `optimize!(model)` to solve the LP:

In [3]:
optimize!(power)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
3 rows, 2 cols, 6 nonzeros
3 rows, 2 cols, 6 nonzeros
Objective function is integral with scale 0.2

Solving MIP model with:
   3 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   6 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   90              inf                  inf        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      130
  Dual bound        130
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    130 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (tota

We can use the function `value.()` to get the value of each variable, and `objective_value()` to get the total objective value.

In [4]:
println("# Coal Plants: ", value.(x))
println("# Wind Farms: ", value.(y))
println("Cost: ", objective_value(power))

# Coal Plants: 4.0
# Wind Farms: 6.0
Cost: 130.0


We can also use the JuMP function `solution_summary` to see more details of the <a href="https://jump.dev/JuMP.jl/stable/manual/solutions/" target="_blank">solution</a>:

In [ ]:
solution_summary(power)

### GenX

Let's optimize the GenX model created in the last Tutorial. To do so, we'll create the inputs for `generate_model` and run it. 

In [5]:
using GenX

In [6]:
case = joinpath("Example_Systems_Tutorials/SmallNewEngland/OneZone") 

genx_settings = GenX.get_settings_path(case, "genx_settings.yml");
setup = GenX.configure_settings(genx_settings);
settings_path = GenX.get_settings_path(case)

### Create TDR_Results
TDRpath = joinpath(case, setup["TimeDomainReductionFolder"])

if setup["TimeDomainReduction"] == 1
    GenX.prevent_doubled_timedomainreduction(case)
    if !GenX.time_domain_reduced_files_exist(TDRpath)
        println("Clustering Time Series Data (Grouped)...")
        GenX.cluster_inputs(case, settings_path, setup)
    else
        println("Time Series Data Already Clustered.")
    end
end

OPTIMIZER =  GenX.configure_solver(setup["Solver"], settings_path);

inputs = GenX.load_inputs(setup, case)

Configuring Settings
Time Series Data Already Clustered.
Reading Input CSV Files
Network.csv Successfully Read!
Load_data.csv Successfully Read!
Fuels_data.csv Successfully Read!
Generators_data.csv Successfully Read!
Generators_variability.csv Successfully Read!
Validating time basis
Capacity_reserve_margin.csv Successfully Read!
Minimum_capacity_requirement.csv Successfully Read!
Maximum_capacity_requirement.csv Successfully Read!
Energy_share_requirement.csv Successfully Read!
CO2_cap.csv Successfully Read!
CSV Files Successfully Read In From Example_Systems_Tutorials/SmallNewEngland/OneZone


Dict{Any, Any} with 66 entries:
  "Z"                   => 1
  "LOSS_LINES"          => [1]
  "RET_CAP_CHARGE"      => Int64[]
  "pC_D_Curtail"        => [50.0]
  "dfGen"               => 4×68 DataFrame…
  "pTrans_Max_Possible" => [2.95]
  "pNet_Map"            => [1.0;;]
  "omega"               => [4.01099, 4.01099, 4.01099, 4.01099, 4.01099, 4.0109…
  "RET_CAP_ENERGY"      => [4]
  "RESOURCES"           => String31["natural_gas_combined_cycle", "solar_pv", "…
  "COMMIT"              => [1]
  "pMax_D_Curtail"      => [1]
  "STOR_ALL"            => [4]
  "THERM_ALL"           => [1]
  "dfCO2CapZones"       => [1;;]
  "REP_PERIOD"          => 11
  "MinCapReq"           => [5.0, 10.0, 6.0]
  "STOR_LONG_DURATION"  => Int64[]
  "dfCapRes"            => [0.156;;]
  "STOR_SYMMETRIC"      => [4]
  "VRE"                 => [2, 3]
  "RETRO"               => Int64[]
  "THERM_COMMIT"        => [1]
  "TRANS_LOSS_SEGS"     => 1
  "H"                   => 168
  ⋮                     => ⋮

In [7]:
EP = GenX.generate_model(setup,inputs,OPTIMIZER)

Discharge Module
Non-served Energy Module
Investment Discharge Module
Unit Commitment Module
Emissions Module (for CO2 Policy modularization
Dispatchable Resources Module
Storage Resources Module
Storage Investment Module
Storage Core Resources Module
Storage Resources with Symmetric Charge/Discharge Capacity Module
Thermal (Unit Commitment) Resources Module
C02 Policies Module
Energy Share Requirement Policies Module
Capacity Reserve Margin Policies Module
Minimum Capacity Requirement Module
Maximum Capacity Requirement Module


A JuMP Model
Minimization problem with:
Variables: 18492
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 5544 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 7398 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 27730 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 18490 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: cCO2Emissions_systemwide, cCapacityResMargin, cESRShare, cMaxCap, cMaxCapEnergy, cMaxCapEnergyDuration, cMaxNSE, cMaxRetCommit, cMaxRetEnergy, cMaxRetNoCommit, cMinCap, cMinCapEnergy, cMinCapEnergyDuration, cNSEPerSeg, cPowerBalance, cSoCBalInterior, cSoCBalStart, cZoneMaxCapReq, cZoneMinCapReq, eCFix, eCFixEnergy, eCNSE, eCStart, eCVar_in, eCVar_out, eCapResMarBalance, eCapResMarBalanceStor, eCapResMarBalanceThermal, eCapResMarBalanceVRE, eEL

The function `solve_model(model, setup)` uses `optimize` to optimize the model:

In [8]:
solution = optimize!(EP)
#solution = @timed GenX.solve_model(EP,setup)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
36011 rows, 17528 cols, 136561 nonzeros
31479 rows, 15157 cols, 125413 nonzeros
Presolve : Reductions: rows 31479(-9193); columns 15157(-3335); elements 125413(-40801)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.9393262080e+04 Pr: 2222(11537.4) 0s
       6460     2.1173188918e+04 Pr: 0(0); Du: 0(12.551) 0s
       6460     2.1173188918e+04 Pr: 0(0); Du: 0(12.551) 0s
       6460     2.1173188918e+04 Pr: 0(0); Du: 0(12.551) 0s
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
       6460     2.1173317136e+04 Pr: 937(2832.23); Du: 0(2.69444) 0s
       8668     2.1228832736e+04 Pr: 0(0); Du: 0(8.59055) 0s
       8668     2.1228832736e+04 Pr: 0(0); Du: 0(8.59055) 0s
Solving the original LP from the solution after postsolve
Using EKK dual simplex solver - serial
  It

(value = nothing, time = 15.331503375, bytes = 52479066, gctime = 0.014805625, gcstats = Base.GC_Diff(52479066, 61, 0, 520417, 176, 87, 14805625, 1, 0))

In [9]:
objective_value(EP)

9776.57688838726

Note that regardless of solver settings, the folder `TDR_Results` stays the same, so we won't change it throughout the tutorial.

### Solver Settings

Though solving the model relies only on `optimize`, there are a number of ways to change the way in which the model is optimized. This section goes over solver parameters and how they affect the model solution.

In [10]:
using YAML

#### The HiGHS Solver

In the example files, the solver <a href="https://highs.dev" target="_blank">HiGHS</a>. HiGHS is freely available for all to use. Other solvers, such as  <a href="https://www.gurobi.com" target="_blank">Gurobi</a>, are available for free for academics. For the purpose of this tutorial, we will be focusing on HiGHS. 

To set the solver preferences, go into the settings folder of your case and select the YAML file of the solver you're using.

In [11]:
settings_folder = cd(readdir,joinpath(case,"Settings")) # Print Settings folder

8-element Vector{String}:
 "cbc_settings.yml"
 "clp_settings.yml"
 "cplex_settings.yml"
 "genx_settings.yml"
 "gurobi_settings.yml"
 "highs_settings.yml"
 "scip_settings.yml"
 "time_domain_reduction_settings.yml"

In [12]:
highs_settings = YAML.load(open(joinpath(case,"Settings/highs_settings.yml")))

Dict{Any, Any} with 6 entries:
  "Method"        => "choose"
  "Feasib_Tol"    => 0.1
  "run_crossover" => "on"
  "TimeLimit"     => 1.0e23
  "Optimal_Tol"   => 0.1
  "Pre_Solve"     => "choose"

The function <a href="https://genxproject.github.io/GenX/dev/solver_configuration/#Configuring-HiGHS" target="_blank">`configure_highs`</a> in `src/configure_solver` contains a list of default settings for the HiGHS solver

<img src="files/highs_defaults.png" style="width: auto; height: 500px" align="left">

There are about 80, so we'll only focus on a few for now. In most cases, you can leave the other settings on default. 

The default settings are combined with the settings you specify in `highs_settings.yml` in `configure_highs`, which is called from `configure_solver` in `run_genx_case_simple` right before the model is generated.

#### Feasibility Tolerance

The parameters `Feasib_Tol` and `Optimal_Tol` represent the feasibility of the primal and dual functions respectively. Without going into too much detail, a  <a href="https://en.wikipedia.org/wiki/Duality_(optimization)" target="_blank">__dual function__</a> is an analagous formulation of the original ("primal") function whose objective value acts as a lower bound to the primal function. The objective value of the primal function is then the upper bound of the dual function. HiGHS will solve the dual and primal at each time step, then terminate when the solutions of the two are within a certain tolerance range. For more information on how this works specifically in HiGHS, see the  <a href="https://ergo-code.github.io/HiGHS/dev/terminology/" target="_blank">HiGHS documentaion</a>. 

If we decrease the tolerance parameters, the objective value becomes closer to the "true" optimal value.

In [16]:
# Change tolerance, generate and solve model`
tols = [1e-7,1e-5,1e-4,1e-3,1e-2,1e-1]
OV = [0.0,0.0,0.0,0.0,0.0,0.0]
#highs_settings["Method"] = "ipm"
#YAML.write_file(joinpath(case,"Settings/highs_settings.yml"), highs_settings)

for i in range(1,length(tols))
    println(" ")
    println("----------------------------------------------------")
    println("Iteration ",i)
    println("Tolerance = ",tols[i])
    println("----------------------------------------------------")
    highs_settings["Feasib_Tol"] = tols[i]
    highs_settings["Optimal_Tol"] = tols[i]
    YAML.write_file(joinpath(case,"Settings/highs_settings.yml"), highs_settings)
    OPTIMIZER1 = GenX.configure_solver(setup["Solver"], settings_path);
    EP = GenX.generate_model(setup,inputs,OPTIMIZER)
    solution = GenX.solve_model(EP,setup)
    OV[i] = objective_value(EP)
end



 
----------------------------------------------------
Iteration 1
Tolerance = 1.0e-7
----------------------------------------------------
Discharge Module
Non-served Energy Module
Investment Discharge Module
Unit Commitment Module
Emissions Module (for CO2 Policy modularization
Dispatchable Resources Module
Storage Resources Module
Storage Investment Module
Storage Core Resources Module
Storage Resources with Symmetric Charge/Discharge Capacity Module
Thermal (Unit Commitment) Resources Module
C02 Policies Module
Energy Share Requirement Policies Module
Capacity Reserve Margin Policies Module
Minimum Capacity Requirement Module
Maximum Capacity Requirement Module
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
36011 rows, 17528 cols, 136561 nonzeros
34548 rows, 16069 cols, 136332 nonzeros
Presolve : Reductions: rows 34548(-6124); columns 16069(-2423); elements 136332(-29882)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iterat

Using the smallest tolerance as our base, we can see the error as the tolerance increases:

In [17]:
[tols[2:end] abs.(OV[2:end] .- OV[1])]

LoadError: DimensionMismatch: vectors must have same lengths

In [ ]:
using Plots
using Plotly

In [ ]:
# Plot the run time as a function of the tolerance
plotlyjs()
Plots.scatter(tols[2:end], abs.(OV[2:end] .- OV[1]), hover = tols[2:end], legend=false,
                ylabel="Error", xlabel="Tolerance", title="Tolerance vs Error",size=(980,450))

### PreSolve

In optimization, presolve is a stage at the beginning of the solver in which the problem is simplified to remove redunant constraints and otherwise simplify the problem before the optimization itself begins. The default for presolve in GenX is "choose", allowing the solver to use presolve only if it will reduce computation time. 

Let's try setting presolve to off and on, then compare computation times.

In [ ]:
# First, set tolerances back to original
highs_settings["Feasib_Tol"] = 1e-5
highs_settings["Optimal_Tol"] = 1e-5
YAML.write_file(joinpath(case,"Settings/highs_settings.yml"), highs_settings)
    

In [ ]:
highs_settings["Pre_Solve"] = "off"
YAML.write_file(joinpath(case,"Settings/highs_settings.yml"), highs_settings)
OPTIMIZER2 = GenX.configure_solver(setup["Solver"], settings_path);
EP2 = GenX.generate_model(setup,inputs,OPTIMIZER2)

In [ ]:
solution2 = @elapsed GenX.solve_model(EP2,setup)

In [ ]:
highs_settings["Pre_Solve"] = "on"
YAML.write_file(joinpath(case,"Settings/highs_settings.yml"), highs_settings)
OPTIMIZER3 = GenX.configure_solver(setup["Solver"], settings_path);
EP3 = GenX.generate_model(setup,inputs,OPTIMIZER3)

In [ ]:
solution3 = @elapsed GenX.solve_model(EP3,setup)

As we can see, the runtime with PreSolve is shorter, and would be even shorter for a larger system. However, it could introduce numerical inaccuracies. If you find the model is struggling to converge, try turn PreSolve off.

In [ ]:
# Write PreSolve back to choose
highs_settings["Pre_Solve"] = "choose"
YAML.write_file(joinpath(case,"Settings/highs_settings.yml"), highs_settings)

## Infeasibility

In some cases, your model may not be able to return a value. This happens when no value can be found that satisfies all constraints. To see this, let's go back to our simple example and change one of the parameters to break the model.

\begin{align}
& \min 10 x + 15 y &\text{Objective function (cost)}\\ 
& \text{s.t.} & \\
& x + y \geq 10 &\text{Grid Demand}\\
& 55x + 70y \leq \ 1000 &\text{Construction constraint}\\
& 40 x + 5 y \leq 200 &\text{Emissions constraint} \\
& 7 x + 30 y \geq 800 &\textbf{New Constraint} \\
& x, y \geq 0 &\text{Non-negativity constraints}\\
\end{align}

In [ ]:
@constraint(power, new, 7x + 30y >= 100)

In [ ]:
print(power)

In [ ]:
optimize!(power)

In this case, the infeasibility was detected on the presovle since it's clear no solution would fit within all constraints. For information on how to debug an infeasible solution, see the <a href="https://jump.dev/JuMP.jl/stable/tutorials/getting_started/debugging/#Common-sources-4" target="_blank">JuMP documentaion</a>. 

In [19]:
setup

Dict{Any, Any} with 24 entries:
  "NetworkExpansion"                        => 0
  "TimeDomainReductionFolder"               => "TDR_Results"
  "EnableJuMPStringNames"                   => false
  "Trans_Loss_Segments"                     => 1
  "ModelingtoGenerateAlternativeSlack"      => 0.1
  "Solver"                                  => "HiGHS"
  "Reserves"                                => 0
  "MultiStage"                              => 0
  "OverwriteResults"                        => 0
  "ModelingToGenerateAlternatives"          => 0
  "MaxCapReq"                               => 1
  "MinCapReq"                               => 1
  "CO2Cap"                                  => 2
  "WriteShadowPrices"                       => 1
  "ModelingToGenerateAlternativeIterations" => 3
  "ParameterScale"                          => 1
  "EnergyShareRequirement"                  => 1
  "PrintModel"                              => 0
  "TimeDomainReduction"                     => 1
  "CapacityRe

In [ ]:
value.(EP[:eTotalCap])

In [ ]:
inputs["dfGen"][:,:Resource_Type]

In [ ]:
setup["CO2Cap"]=1;

In [ ]:
inputs["dfCO2Cap"].CO_2_Max_tons_MWh_1.=0.0

In [ ]:
EP = 